# Exercise: Train your own transformer!

EXPLANAINTION: https://www.youtube.com/watch?v=kE6leQAjTyA

In this exercise, we will construct and train a minified GPT implementation. GPT refers to the "Generative Pre-trained Transformers" from OpenAI, originally described in ["Improving language understanding with unsupervised learning"](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf). This specific GPT implementation is heavily inspired by the [minGPT implementation](https://github.com/karpathy/minGPT) provided by [Andrej Karpathy](https://github.com/karpathy/).

There are three important PyTorch modules here:
* `MultiHeadSelfAttention`: a self-attention implementation which will be provided for you;
* `Block`: a transformer block which is repeated n_layer times in a GPT model;
* `GPT`: the full GPT model itself, including intial embeddings, the GPT blocks, and the token decoding logic.


The `GPT` module uses the `Block` module, which in turn uses the `MultiHeadSelfAttention` module.
```                                   
    ┌────────────────────────┐     
    │          GPT           │     
    └────────────────────────┘     
                ▲                  
    ┌───────────┴────────────┐     
    │         Block          │     
    └────────────────────────┘     
                ▲                  
    ┌───────────┴────────────┐     
    │ MultiHeadSelfAttention │     
    └────────────────────────┘     

```

## Step: Import and show MultiHeadSelfAttention

In [6]:
from common_exercise_2_create_your_own_gpt_model_starter import GPTConfig, MultiHeadSelfAttention

# Let's use a placeholder config to show how the attention layer works
config = GPTConfig(
    vocab_size=10,
    n_layer=3,
    n_embd=12,
    n_head=4,
    block_size=5,
)

attention = MultiHeadSelfAttention(config)

print(attention)

/home/tumaaa/work/repos/udacity_nano_degree_generative_ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MultiHeadSelfAttention(
  (key): Linear(in_features=12, out_features=12, bias=True)
  (query): Linear(in_features=12, out_features=12, bias=True)
  (value): Linear(in_features=12, out_features=12, bias=True)
  (attn_drop): Dropout(p=0.1, inplace=False)
  (resid_drop): Dropout(p=0.1, inplace=False)
  (proj): Linear(in_features=12, out_features=12, bias=True)
)


## Step: Create the Transformer Block

Now we are going to create the GPT model using the `MultiHeadSelfAttention` module. Please fill in the sections marked `TODO`.

In this cell, we are going to implement what is called a residual connection, which takes the form:

```
x := x + MultiHeadSelfAttention(LayerNorm(x)) + MLP(LayerNorm(x))
```

In [19]:
import torch.nn as nn


class Block(nn.Module):
    """an unassuming Transformer block"""

    # === EXERCISE PART 1 START: CONSTRUCT A TRANSFORMER BLOCK ===
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)

        # TODO: Instantiate the MultiHeadSelfAttention module
        self.attn = MultiHeadSelfAttention(config=config)

        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.resid_pdrop),
        )

    def forward(self, x):
        # TODO: implement a residual connection of the following form

        # x := x + MultiHeadSelfAttention(LayerNorm(x)) + MLP(LayerNorm(x))
        x = x + self.attn(self.ln1(x)) + self.mlp(self.ln2(x))

        # Hint: MultiHeadSelfAttention, LayerNorm, and MLP were all instantiated in __init__
        # and are available as properties of self, e.g. self.attn

        return x

    # === EXERCISE PART 1 END: CONSTRUCT A TRANSFORMER BLOCK ===


# Check that the block instantiates
block = Block(config)
block

Block(
  (ln1): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
  (attn): MultiHeadSelfAttention(
    (key): Linear(in_features=12, out_features=12, bias=True)
    (query): Linear(in_features=12, out_features=12, bias=True)
    (value): Linear(in_features=12, out_features=12, bias=True)
    (attn_drop): Dropout(p=0.1, inplace=False)
    (resid_drop): Dropout(p=0.1, inplace=False)
    (proj): Linear(in_features=12, out_features=12, bias=True)
  )
  (mlp): Sequential(
    (0): Linear(in_features=12, out_features=48, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=48, out_features=12, bias=True)
    (3): Dropout(p=0.1, inplace=False)
  )
)

In [20]:
# Let's check some aspects of the block
block = Block(config)
assert isinstance(block.attn, MultiHeadSelfAttention)

## Step: Let's construct the GPT module
It's time to put it all together and make our GPT PyTorch model. As before, fill in the `TODO` sections.

In [29]:
import torch
import torch.nn.functional as F


class GPT(nn.Module):
    """the full GPT language model, with a context size of block_size"""

    # === EXERCISE PART 2 START: COMPLETE THE GPT MODEL ===

    def __init__(self, config):
        super().__init__()

        # input embedding stem
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.embd_pdrop)

        # transformer
        # TODO: Instantiate a sequence of N=config.n_layer transformer blocks.
        # Hint: use nn.Sequential to chain N instances of the Block module.
        self.blocks = nn.Sequential(*[Block(config)]*config.n_layer)

        # decoder head
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size
        self.apply(self._init_weights)

        print(
            "number of parameters: {}".format(sum(p.numel() for p in self.parameters()))
        )

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def get_block_size(self):
        return self.block_size

    def forward(self, idx, targets=None):
        b, t = idx.size()
        assert t <= self.block_size, "Cannot forward, model block size is exhausted."

        # Create token embeddings and add positional embeddings
        token_embeddings = self.tok_emb(idx)  # each index maps to a (learnable) vector
        position_embeddings = self.pos_emb[
            :, :t, :
        ]  # each position maps to a (learnable) vector
        x = self.drop(token_embeddings + position_embeddings)

        # TODO: Pass the embeddings through the transformer blocks, created previously in __init__
        x = self.blocks(x)

        # Decode the output of the transformer blocks
        x = self.ln_f(x)
        logits = self.head(x)

        # If we are given some desired targets also calculate the loss, e.g. during training
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1
            )
        else:
            loss = None

        return logits, loss

    # === EXERCISE PART 2 END: COMPLETE THE GPT MODEL ===

    @torch.no_grad()
    def generate(
        self, idx, max_new_tokens, temperature=1.0, top_k=None, stop_tokens=None
    ):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """

        # === EXERCISE PART 3 START: COMPLETE THE GENERATION LOGIC ===
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = (
                idx if idx.size(1) <= self.block_size else idx[:, -self.block_size :]
            )

            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)

            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature

            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float("Inf")

            # TODO: apply softmax to convert logits to (normalized) probabilities
            # using F.softmax. Remember the dim=-1 parameter.
            probs = F.softmax(logits, dim=-1)

            # TODO: sample from the distribution (if top_k=1 this is equivalent to greedy sampling)
            # using torch.multinomial. You only need to sample a single token.
            idx_next = torch.multinomial(probs, num_samples=1)

            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

            # stop prediction if we produced a stop token
            if stop_tokens is not None and idx_next.item() in stop_tokens:
                return idx
        # === EXERCISE PART 3 END: COMPLETE THE GENERATION LOGIC ===

        return idx

In [30]:
# Let's check this model runs real quickly


model = GPT(config)
input_seq = torch.tensor([[1, 2, 3]])
output_seq = model.generate(input_seq, max_new_tokens=30)

# Check the generated sequence shape
assert output_seq.shape == (1, 33)

print("input sequence:", input_seq.tolist()[0])
print("output sequence:", output_seq.tolist()[0])

print("Success!")

number of parameters: 2208
input sequence: [1, 2, 3]
output sequence: [1, 2, 3, 3, 5, 6, 3, 2, 5, 8, 8, 0, 0, 3, 5, 6, 9, 7, 9, 5, 5, 9, 8, 4, 5, 5, 5, 7, 1, 1, 7, 6, 1]
Success!


## Step: Load a dataset

We will now train our GPT model on a dataset consisting of one- to three-digit addition problems, e.g.

```
111+222=3+30+300=333
```

We break the addition up into two steps (first `=3+30+300` and then finally `=333`) to help the model train more quickly and successfully.

In [32]:
# Run this cell to create a dataset. No modifications are needed.

import numpy as np
from common_exercise_2_create_your_own_gpt_model_starter import AdditionDataset, CharacterTokenizer

BLOCK_SIZE = 32

tokenizer = CharacterTokenizer(
    characters=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", " ", "="],
    model_max_length=BLOCK_SIZE,
)

dataset = AdditionDataset(
    tokenizer=tokenizer,
    block_size=BLOCK_SIZE,
    numbers=list(range(0, 1000, 2)),
    include_intermediate_steps=True,
)

for ix in [11177, 22222]:
    x, y = dataset[ix]

    print(f"=== Example {ix} ===")

    np.set_printoptions(linewidth=999)
    print(f"x = {x.numpy()}")
    print(f"y = {y.numpy()}")

    # show lengths
    print(f"x length = {len(x)}")
    print(f"y length = {len(y)}")

    # print x decoded
    x = tokenizer.decode(x, skip_special_tokens=True)
    print(f"x decoded = {x}")

    # print y decoded, replacing the -1 token with _
    num_unknowns = y.tolist().count(-1)
    y = tokenizer.decode(y[num_unknowns:], skip_special_tokens=True)
    print(f"y decoded = {'_'*(num_unknowns-1)}{y}")

=== Example 11177 ===
x = [ 0 11 11 17 10 12 11 19 15 17 16  7 17 10  7  7 19 10 16 15  1  4  4  4  4  4  4  4  4  4  4  4]
y = [-1 -1 -1 -1 -1 -1 -1 15 17 16  7 17 10  7  7 19 10 16 15  1  4  4  4  4  4  4  4  4  4  4  4  4]
x length = 32
y length = 32
x decoded = 44+354=8+90+300=398
y decoded = ______8+90+300=398
=== Example 22222 ===
x = [ 0 15 15 17 11 11 11 19  9 17 10  7 17 12  7  7 19 12 10  9  1  4  4  4  4  4  4  4  4  4  4  4]
y = [-1 -1 -1 -1 -1 -1 -1  9 17 10  7 17 12  7  7 19 12 10  9  1  4  4  4  4  4  4  4  4  4  4  4  4]
x length = 32
y length = 32
x decoded = 88+444=2+30+500=532
y decoded = ______2+30+500=532


Let's take a minute to examine our dataset.

* What do you notice about the x, the input, and y the target?
* Are they the same length?
* What can you say about the alignment of the sequences?
* What else is different between them?

# Step: Train the model!

Now we will train a small GPT model using this dataset. Along the way we should see how the model's performance improves on real data.

In [33]:
# Instantiate the model and a trainer. No modifications are needed.

from common_exercise_2_create_your_own_gpt_model_starter import GPTConfig, Trainer, TrainerConfig

# instantiate a mini-GPT type model
model_config = GPTConfig(
    vocab_size=tokenizer.vocab_size,
    block_size=BLOCK_SIZE,
    n_layer=3,
    n_head=3,
    n_embd=48,
)

model = GPT(model_config)

# create a trainer
train_config = TrainerConfig(
    max_epochs=1,
    batch_size=1000,
    learning_rate=4e-3,
)

trainer = Trainer(model, dataset, train_config)

# Print the device the trainer will use (cpu, gpu, ...)
print(f"Using device: {trainer.device}")

number of parameters: 31824
Using device: cpu


In [36]:
# Let's see the performance on real data before training. No modifications are needed.
from common_exercise_2_create_your_own_gpt_model_starter import show_examples

show_examples(model, dataset, tokenizer, trainer.device, top_k=1, temperature=1.0)

❌ input->output: [CLS]550+84= 👉 07[UNK]22[MASK][UNK]52762[MASK]2[UNK]5[PAD][PAD]2[MASK][UNK]5[PAD][MASK][UNK]1115 =[UNK] expected: 634, got: None
❌ input->output: [CLS]558+504= 👉 02[MASK][UNK]5[PAD][PAD]27[UNK][SEP] expected: 1062, got: None
❌ input->output: [CLS]278+88= 👉 862[MASK]27[UNK]02[MASK]2[MASK][UNK]7[UNK]02[MASK] =02[MASK][UNK] =02[MASK][UNK] = expected: 366, got: 2
❌ input->output: [CLS]422+740= 👉 02[MASK][UNK]02[MASK]2[MASK]962[MASK] 02[MASK] =[CLS] expected: 1162, got: None
❌ input->output: [CLS]760+660= 👉 [CLS] expected: 1420, got: None
❌ input->output: [CLS]256+630= 👉 [CLS] expected: 886, got: None
❌ input->output: [CLS]120+672= 👉 0[MASK]2[MASK][UNK]02[MASK][UNK]27[UNK]2[MASK][UNK][UNK]02[UNK]5 =[UNK]027[UNK]5[PAD][MASK] 0 expected: 792, got: None
❌ input->output: [CLS]94+418= 👉 02[MASK] 07[UNK]0[MASK][UNK]07[UNK]0[MASK] =0[MASK] =02[MASK][UNK] 3 =0[PAD]3 expected: 512, got: 0
❌ input->output: [CLS]464+796= 👉 08[MASK]2[MASK][UNK]0[MASK]5027[UNK]0[MASK][UNK]1 =02[MASK][UN

In [ ]:
# Let's train the model for an epoch, and see the performance again.
# We will repeat this a few times to see the model improve.
# No modifications are needed.

# Note, depending on your hardware, you may need to reduce the batch size
# if you get any out-of-memory errors.

for _ in range(6):
    trainer.train()  # train just one epoch each time
    show_examples(model, dataset, tokenizer, trainer.device, top_k=3, temperature=1.0)

epoch 1 iter 249: train loss 0.27415. lr 4.000000e-03: 100%|██████████| 250/250 [02:40<00:00,  1.55it/s]


❌ input->output: [CLS]550+84= 👉 2+40+700=720[SEP] expected: 634, got: 720
❌ input->output: [CLS]558+504= 👉 0+0+600+1000=1600[SEP] expected: 1062, got: 1600
❌ input->output: [CLS]278+88= 👉 8+30+300=338[SEP] expected: 366, got: 338
❌ input->output: [CLS]422+740= 👉 4+40+400+1000=1444[SEP] expected: 1162, got: 1444
❌ input->output: [CLS]760+660= 👉 2+60+200+1000=1262[SEP] expected: 1420, got: 1262
❌ input->output: [CLS]256+630= 👉 4+0+600=604[SEP] expected: 886, got: 604
❌ input->output: [CLS]120+672= 👉 4+40+500+1000=1544[SEP] expected: 792, got: 1544
❌ input->output: [CLS]94+418= 👉 4+40+400+1440=1404[SEP] expected: 512, got: 1404
❌ input->output: [CLS]464+796= 👉 4+60+0+1000=1064[SEP] expected: 1260, got: 1064
❌ input->output: [CLS]624+106= 👉 4+40+400+1000=1444[SEP] expected: 730, got: 1444

Correct: 0 out of 10: 0.0%


epoch 1 iter 249: train loss 0.16108. lr 4.000000e-03: 100%|██████████| 250/250 [02:46<00:00,  1.50it/s]


❌ input->output: [CLS]550+84= 👉 4+50+500=554[SEP] expected: 634, got: 554
❌ input->output: [CLS]558+504= 👉 2+90+0+1000=1092[SEP] expected: 1062, got: 1092
❌ input->output: [CLS]278+88= 👉 6+0+900=906[SEP] expected: 366, got: 906
❌ input->output: [CLS]422+740= 👉 2+50+200+1000=1252[SEP] expected: 1162, got: 1252
❌ input->output: [CLS]760+660= 👉 0+50+300+1000=1350[SEP] expected: 1420, got: 1350
❌ input->output: [CLS]256+630= 👉 6+70+800=876[SEP] expected: 886, got: 876
❌ input->output: [CLS]120+672= 👉 2+60+700=762[SEP] expected: 792, got: 762
❌ input->output: [CLS]94+418= 👉 4+40+500=544[SEP] expected: 512, got: 544
❌ input->output: [CLS]464+796= 👉 0+50+200+1000=1150[SEP] expected: 1260, got: 1150
❌ input->output: [CLS]624+106= 👉 0+50+700=750[SEP] expected: 730, got: 750

Correct: 0 out of 10: 0.0%


epoch 1 iter 249: train loss 0.07180. lr 4.000000e-03: 100%|██████████| 250/250 [02:50<00:00,  1.46it/s]


✅ input->output: [CLS]550+84= 👉 4+30+600=634[SEP]
✅ input->output: [CLS]558+504= 👉 2+60+0+1000=1062[SEP]
❌ input->output: [CLS]278+88= 👉 6+60+200+1000=1206[SEP] expected: 366, got: 1206
❌ input->output: [CLS]422+740= 👉 2+60+200+1000=1262[SEP] expected: 1162, got: 1262
❌ input->output: [CLS]760+660= 👉 6+20+400+1000=1426[SEP] expected: 1420, got: 1426
✅ input->output: [CLS]256+630= 👉 6+80+800=886[SEP]
✅ input->output: [CLS]120+672= 👉 2+90+700=792[SEP]
❌ input->output: [CLS]94+418= 👉 2+40+500=542[SEP] expected: 512, got: 542
❌ input->output: [CLS]464+796= 👉 0+60+100+1000=1160[SEP] expected: 1260, got: 1160
✅ input->output: [CLS]624+106= 👉 0+30+700=730[SEP]

Correct: 5 out of 10: 50.0%


epoch 1 iter 7: train loss 0.09506. lr 4.000000e-03:   3%|▎         | 8/250 [00:05<02:47,  1.44it/s]

In [ ]:
# Let's evaluate on 30 examples. No modifications are needed.

show_examples(
    model, dataset, tokenizer, trainer.device, top_k=1, temperature=1.0, max_num=30
)

As the above cell runs, ask yourself the following questions:
* How quickly is the loss decreasing and what do you notice about the structure of the text generation as the loss decreases?
* What is the train loss when you start to observe correct answers?
* What is the train loss when you start to see all correct answers?
* If you continue training after all answers are correct, might you see more incorrect answers?

## Congrats!

Congrats on training your own GPT model! Give yourself a hand! 🙌